<a href="https://colab.research.google.com/github/Angel-dash/CG_1/blob/main/diabetes_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies

In [362]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
import re
import csv

In [363]:
# Read data from file
with open('data.txt', 'r') as file:
    lines = file.readlines()

In [364]:
print(lines)

['Aaron Abernathy is 70 years old Male. Insulin is Up. Pt. has Diabetes with other specified manifestations- type II. Patient is on diabetes meds. Pt. got readmitted in <30 days.\n', 'Aaron Barker is 70 years old Male. Patient is on diabetes meds. Pt. has Diabetes with other specified manifestations- type 1. Patient consulted Family/GeneralPractice specialist. Pt. got readmitted in >30 days. Insulin is Up.\n', 'Aaron Carder is 30 years old Male. Insulin is unknown. His  lab work was conducted. A1c result is >8. Patient is taking metformin, glyburide. Patient is on diabetes meds. Pt. has Diabetes with other specified manifestations- type II.\n', 'Aaron Clayton is 80 years old Male. Pt. has Diabetes with other specified manifestations- type II. Patient consulted Family/GeneralPractice specialist. Insulin is Down. Patient is on diabetes meds.\n', 'Aaron Fallis is 50 years old Male. Pt. has Diabetes with other specified manifestations- type II. Patient is taking metformin, glimepiride, ros

In [365]:
# Initialize empty lists to store patient information
names = []
ages = []
genders = []
diabetes_types = []
insulin_status = []
readmission_statuses = []
medications = []
a1c_levels = []



# Process each patient record
for line in lines:
    line = line.strip()
    if line:
        parts = line.split('. ')
        patient_info = parts[0].split(' is ')
        name = patient_info[0]
        age_gender = patient_info[1].split(' years old ')
        age = int(age_gender[0])
        gender = age_gender[1]
        other_info = parts[1:]

        insulin = 'Unknown'
        medication = 'Not on diabetes meds'
        diabetes_type = 'Unknown'
        readmission_status = 'Unknown'
        a1c_level = 'Unknown'

        for info in other_info:
            if info.startswith('Insulin'):
                insulin = info.split(' is ')[1]
            elif info.startswith('Patient is on diabetes meds'):
                medication = 'Diabetes meds'
            elif 'type 1' in info:
                diabetes_type = 'Type I'
            elif 'type II' in info:
                diabetes_type = 'Type II'
            elif info.startswith('Pt. got readmitted'):
                readmission_status = info.split(' in ')[1]
            elif info.startswith('A1c result'):
                a1c_level = info.split(' is ')[1]

        names.append(name)
        ages.append(age)
        genders.append(gender)
        diabetes_types.append(diabetes_type)
        insulin_status.append(insulin)
        readmission_statuses.append(readmission_status)
        medications.append(medication)
        a1c_levels.append(a1c_level)

# Create a DataFrame from the collected data
data = {
    'Patient Name': names,
    'Age': ages,
    'Gender': genders,
    'Output': diabetes_types,
    'Insulin Status': insulin_status,
    'Readmitted Status': readmission_statuses,
    'Medications': medications,
    'A1C Level': a1c_levels
}
db = pd.DataFrame(data)




In [366]:
print(db)

         Patient Name  Age Gender   Output Insulin Status Readmitted Status  \
0     Aaron Abernathy   70   Male  Type II             Up           Unknown   
1        Aaron Barker   70   Male   Type I            Up.           Unknown   
2        Aaron Carder   30   Male  Type II        unknown           Unknown   
3       Aaron Clayton   80   Male  Type II           Down           Unknown   
4        Aaron Fallis   50   Male  Type II        Steady.           Unknown   
5        Aaron Foster   30   Male  Unknown         Steady           Unknown   
6        Aaron Fowler   70   Male  Type II        unknown           Unknown   
7       Aaron Kennedy   70   Male  Type II        unknown           Unknown   
8        Aaron Landes   50   Male   Type I             Up           Unknown   
9        Aaron Pierre   60   Male  Type II             Up           Unknown   
10      Aaron Shannon   60   Male  Type II            Up.           Unknown   
11       Aaron Torres   60   Male  Type II         S

Data collection and analysis
PIMA diabetes dataset

In [367]:
#loading the diabetes dataset to pandas Datafream
df = pd.read_csv('/content/DataOutput.csv')

In [368]:
#printing the first five rows of the dataset
df.head()

,Patient Name,Age,Gender,Insulin Up,Insulin Down,Insulin Steady,Readmitted Status,Medications,A1C Level,Output
0,Aaron Abernathy,70,Male,1,0,0,Unknown,Diabetes meds,8.5,Type II
1,Aaron Barker,70,Male,1,0,0,Unknown,Diabetes meds,8.5,Type I
2,Aaron Carder,30,Male,0,0,0,Unknown,Diabetes meds,7.5,Type II
3,Aaron Clayton,80,Male,0,0,0,Unknown,Diabetes meds,8.5,Type II
4,Aaron Fallis,50,Male,0,0,0,Unknown,Diabetes meds,8.0,Type II


In [369]:
# Create a dictionary of key-value pairs
replacements = {'Type I': 1, 'Type II': 0}

# Replace the values in the `type` column
df['Output'] = df['Output'].replace(replacements)

# Print the DataFrame
print(df)

         Patient Name  Age Gender  Insulin Up  Insulin Down  Insulin Steady  \
0     Aaron Abernathy   70   Male           1             0               0   
1        Aaron Barker   70   Male           1             0               0   
2        Aaron Carder   30   Male           0             0               0   
3       Aaron Clayton   80   Male           0             0               0   
4        Aaron Fallis   50   Male           0             0               0   
5        Aaron Foster   30   Male           0             0               0   
6        Aaron Fowler   70   Male           0             0               0   
7       Aaron Kennedy   70   Male           0             0               0   
8        Aaron Landes   50   Male           0             0               0   
9        Aaron Pierre   60   Male           1             0               0   
10      Aaron Shannon   60   Male           1             0               0   
11       Aaron Torres   60   Male           0       

In [370]:
# Create a dictionary of key-value pairs
replacements = {'Diabetes meds': 1, 'Not on diabetes meds': 0}

# Replace the values in the `type` column
df['Medications'] = df['Medications'].replace(replacements)
A=df
# Print the DataFrame
print(A)

         Patient Name  Age Gender  Insulin Up  Insulin Down  Insulin Steady  \
0     Aaron Abernathy   70   Male           1             0               0   
1        Aaron Barker   70   Male           1             0               0   
2        Aaron Carder   30   Male           0             0               0   
3       Aaron Clayton   80   Male           0             0               0   
4        Aaron Fallis   50   Male           0             0               0   
5        Aaron Foster   30   Male           0             0               0   
6        Aaron Fowler   70   Male           0             0               0   
7       Aaron Kennedy   70   Male           0             0               0   
8        Aaron Landes   50   Male           0             0               0   
9        Aaron Pierre   60   Male           1             0               0   
10      Aaron Shannon   60   Male           1             0               0   
11       Aaron Torres   60   Male           0       

In [371]:
print(df)

         Patient Name  Age Gender  Insulin Up  Insulin Down  Insulin Steady  \
0     Aaron Abernathy   70   Male           1             0               0   
1        Aaron Barker   70   Male           1             0               0   
2        Aaron Carder   30   Male           0             0               0   
3       Aaron Clayton   80   Male           0             0               0   
4        Aaron Fallis   50   Male           0             0               0   
5        Aaron Foster   30   Male           0             0               0   
6        Aaron Fowler   70   Male           0             0               0   
7       Aaron Kennedy   70   Male           0             0               0   
8        Aaron Landes   50   Male           0             0               0   
9        Aaron Pierre   60   Male           1             0               0   
10      Aaron Shannon   60   Male           1             0               0   
11       Aaron Torres   60   Male           0       

In [372]:
print(df)

         Patient Name  Age Gender  Insulin Up  Insulin Down  Insulin Steady  \
0     Aaron Abernathy   70   Male           1             0               0   
1        Aaron Barker   70   Male           1             0               0   
2        Aaron Carder   30   Male           0             0               0   
3       Aaron Clayton   80   Male           0             0               0   
4        Aaron Fallis   50   Male           0             0               0   
5        Aaron Foster   30   Male           0             0               0   
6        Aaron Fowler   70   Male           0             0               0   
7       Aaron Kennedy   70   Male           0             0               0   
8        Aaron Landes   50   Male           0             0               0   
9        Aaron Pierre   60   Male           1             0               0   
10      Aaron Shannon   60   Male           1             0               0   
11       Aaron Torres   60   Male           0       

In [373]:
print(df)

         Patient Name  Age Gender  Insulin Up  Insulin Down  Insulin Steady  \
0     Aaron Abernathy   70   Male           1             0               0   
1        Aaron Barker   70   Male           1             0               0   
2        Aaron Carder   30   Male           0             0               0   
3       Aaron Clayton   80   Male           0             0               0   
4        Aaron Fallis   50   Male           0             0               0   
5        Aaron Foster   30   Male           0             0               0   
6        Aaron Fowler   70   Male           0             0               0   
7       Aaron Kennedy   70   Male           0             0               0   
8        Aaron Landes   50   Male           0             0               0   
9        Aaron Pierre   60   Male           1             0               0   
10      Aaron Shannon   60   Male           1             0               0   
11       Aaron Torres   60   Male           0       

In [374]:
#number of rows and columns in this dataset
df.shape

(55, 10)

In [375]:
#seperating the data and labels
X = df.drop(columns=['Output','Gender','Patient Name','Readmitted Status'], axis=1)

Y=df['Output']

In [376]:
print(X)

    Age  Insulin Up  Insulin Down  Insulin Steady  Medications  A1C Level
0    70           1             0               0            1        8.5
1    70           1             0               0            1        8.5
2    30           0             0               0            1        7.5
3    80           0             0               0            1        8.5
4    50           0             0               0            1        8.0
5    30           0             0               0            1        7.5
6    70           0             0               0            1        8.5
7    70           0             0               0            1        8.5
8    50           0             0               0            1        8.0
9    60           1             0               0            1        8.5
10   60           1             0               0            1        8.5
11   60           0             0               1            1        8.5
12   50           0             0     

In [377]:
print(Y)

0     0
1     1
2     0
3     0
4     0
5     1
6     0
7     0
8     1
9     0
10    0
11    0
12    0
13    1
14    0
15    1
16    0
17    1
18    0
19    1
20    0
21    0
22    0
23    0
24    1
25    0
26    1
27    0
28    1
29    0
30    0
31    0
32    1
33    0
34    0
35    0
36    0
37    1
38    0
39    1
40    0
41    1
42    0
43    0
44    0
45    0
46    1
47    0
48    0
49    0
50    0
51    0
52    0
53    0
54    0
Name: Output, dtype: int64


In [378]:
#getting the statistical measrures of the data
df.describe()

,Age,Insulin Up,Insulin Down,Insulin Steady,Medications,A1C Level,Output
count,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000
mean,49.454545,0.309091,0.072727,0.327273,0.872727,7.918182,0.272727
std,20.038683,0.466378,0.262082,0.473542,0.336350,0.671949,0.449467
min,10.000000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000
25%,40.000000,0.000000,0.000000,0.000000,1.000000,8.000000,0.000000
50%,50.000000,0.000000,0.000000,0.000000,1.000000,8.000000,0.000000
75%,60.000000,1.000000,0.000000,1.000000,1.000000,8.500000,1.000000
max,90.000000,1.000000,1.000000,1.000000,1.000000,8.500000,1.000000


In [379]:
df['Output'].value_counts()

0    40
1    15
Name: Output, dtype: int64

0--> Type 2
1--> Type 1

In [380]:
df.groupby('Output').mean()

<ipython-input-380-9072c0085958>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('Output').mean()


,Age,Insulin Up,Insulin Down,Insulin Steady,Medications,A1C Level
Output,,,,,,
0,54.750000,0.275,0.075000,0.350000,0.875000,8.1125
1,35.333333,0.400,0.066667,0.266667,0.866667,7.4000


In [381]:
print(X)

    Age  Insulin Up  Insulin Down  Insulin Steady  Medications  A1C Level
0    70           1             0               0            1        8.5
1    70           1             0               0            1        8.5
2    30           0             0               0            1        7.5
3    80           0             0               0            1        8.5
4    50           0             0               0            1        8.0
5    30           0             0               0            1        7.5
6    70           0             0               0            1        8.5
7    70           0             0               0            1        8.5
8    50           0             0               0            1        8.0
9    60           1             0               0            1        8.5
10   60           1             0               0            1        8.5
11   60           0             0               1            1        8.5
12   50           0             0     

In [382]:
print(Y)

0     0
1     1
2     0
3     0
4     0
5     1
6     0
7     0
8     1
9     0
10    0
11    0
12    0
13    1
14    0
15    1
16    0
17    1
18    0
19    1
20    0
21    0
22    0
23    0
24    1
25    0
26    1
27    0
28    1
29    0
30    0
31    0
32    1
33    0
34    0
35    0
36    0
37    1
38    0
39    1
40    0
41    1
42    0
43    0
44    0
45    0
46    1
47    0
48    0
49    0
50    0
51    0
52    0
53    0
54    0
Name: Output, dtype: int64


In [383]:
#Data Standarization in particular range so machine learning model can make predection 
#scaler = StandardScaler()

In [384]:
#scaler.fit(X)

In [385]:
#standarized_data=scaler.transform(X)#this will transfrom the data in the same range 

In [386]:
#print(standarized_data)

In [387]:
#X=standarized_data
#Y=df['Output']

In [388]:
print(X) #X will give standarized data
print(Y)# Y wil give label of the outcome i.e diabetic or non-diabetic

    Age  Insulin Up  Insulin Down  Insulin Steady  Medications  A1C Level
0    70           1             0               0            1        8.5
1    70           1             0               0            1        8.5
2    30           0             0               0            1        7.5
3    80           0             0               0            1        8.5
4    50           0             0               0            1        8.0
5    30           0             0               0            1        7.5
6    70           0             0               0            1        8.5
7    70           0             0               0            1        8.5
8    50           0             0               0            1        8.0
9    60           1             0               0            1        8.5
10   60           1             0               0            1        8.5
11   60           0             0               1            1        8.5
12   50           0             0     

Spliting train and test data

In [389]:
X_train, X_test, Y_train, Y_test= train_test_split(X,Y, test_size=0.2,stratify=Y,random_state=2)#train_split_data is already imported from the from sklearn.model_selection import train_test_split
#Y_train contains all the lebel for the X_train data and Y_test contains all the labels for the X_test data
#stratify=Y means that the data will be split in a stratified manner, using Y as the class labels. This ensures that the proportion of each class in the training and test sets is the same as in the original data.
#random_state=2 sets the random seed to ensure that the results are reproducible.

In [390]:
print(X.shape, X_train.shape,X_test.shape)

(55, 6) (44, 6) (11, 6)


Training the model

In [391]:
classifier=svm.SVC(kernel='linear')#here we are loading the support vector model
#A linear kernel is a simple and effective way to separate data that is linearly separable, 
#meaning that a straight line can be drawn to separate the different classes. 
#In the case of diabetes detection using machine learning, it’s possible that the data is linearly separable
# and a linear SVM model can provide good classification performance.

In [392]:
#training the support vector machine classifier
classifier.fit(X_train,Y_train)

SVC(kernel='linear')

Model Evaluation 

Accuracy Score

In [393]:
#accuracy score on the training data
X_train_prediction=classifier.predict(X_train)
training_data_accuracy= accuracy_score(X_train_prediction, Y_train)

In [394]:
print("Accuracy score of the training data: ", training_data_accuracy)

Accuracy score of the training data:  0.7954545454545454


In [395]:
#accuracy score on the test data
X_test_prediction=classifier.predict(X_test)
test_data_accuracy= accuracy_score(X_test_prediction, Y_test)

In [396]:
print("Accuracy sore on test data:",test_data_accuracy)

Accuracy sore on test data: 0.8181818181818182


Making a predective system

In [397]:
input_data = (2,0,0,0,0,5)

#change the input data to a numpy array 
input_data_as_numpy_array=np.asarray(input_data)

#reshape the array as we area predecting for one instance
#if we do not reshape the array then the model will expect to have 764 data which is the total number of dataset
#it will tell the model we are not looking for all the other 764 instances but only for one instances
input_data_reshaped= input_data_as_numpy_array.reshape(1,-1)

#we need to standarized the input data since we are not using the raw data but the standarized input data

#standaraizes the input data
#std_data=scaler.transform(input_data_reshaped)
#print(std_data)

prediction=classifier.predict(input_data_reshaped)
print(prediction)
if (prediction[0]==0):#The prediction gives the values the value in the form of a list and the result is displayed at an index of 0 so
  print("The person is type 2 diabetic")
else:
  print("The person is type 1 diabetic")

[1]
The person is type 1 diabetic


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


Saving the trained model 

In [398]:
import pickle

In [399]:
filename = 'Diabetes_trained_modelv2.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [400]:
# loading the saved model
loaded_model = pickle.load(open('Diabetes_trained_modelv2.sav', 'rb'))

In [401]:
input_data=(1,0,0,0,0,4.5)
#change the input data to a numpy array 
input_data_as_numpy_array=np.asarray(input_data)

#reshape the array as we area predecting for one instance
#if we do not reshape the array then the model will expect to have 764 data which is the total number of dataset
#it will tell the model we are not looking for all the other 764 instances but only for one instances
input_data_reshaped= input_data_as_numpy_array.reshape(1,-1)

#we need to standarized the input data since we are not using the raw data but the standarized input data

#standaraizes the input data
#std_data=scaler.transform(input_data_reshaped)
#print(std_data)

prediction=loaded_model.predict(input_data_reshaped)
print(prediction)
if (prediction[0]==0):#The prediction gives the values the value in the form of a list and the result is displayed at an index of 0 so
  print("The person is type 2 diabetic")
else:
  print("The person is type 1 diabetic")

[1]
The person is type 1 diabetic


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
